# The notebook is structured as follows:

**Import Libraries**: Đây là phần nhập các thư viện và module Python cần thiết để thao tác dữ liệu, trực quan hóa và học máy.

**Load Datasets**: Năm bộ dữ liệu khác nhau liên quan đến doanh số cửa hàng, thông tin cửa hàng, giá dầu, sự kiện lễ hội và giao dịch được tải vào trong bước này.

**Preprocessing**: Bước này chuyển đổi dữ liệu raw thành định dạng dễ tiêu thụ hơn cho các thuật toán học máy. Chúng ta thực hiện một loạt các công việc tiền xử lý như chuyển đổi cột ngày thành đối tượng datetime, mã hóa các đặc trưng hạng mục, kết hợp dữ liệu từ các bộ dữ liệu khác nhau, xử lý giá trị thiếu và tạo ra các đặc trưng mới.

**Model Building & Evaluation**: Ở đây, chúng ta huấn luyện bốn mô hình khác nhau: XGBoost, ARIMA, LSTM và Prophet. Mỗi mô hình được chọn vì điểm mạnh cụ thể trong xử lý dữ liệu chuỗi thời gian. Sau khi huấn luyện, hiệu suất của mỗi mô hình được đánh giá bằng độ đo Mean Squared Error (MSE).

**Submission**: Dựa trên hiệu suất của các mô hình, chúng ta chọn mô hình tốt nhất và sử dụng nó để dự đoán trên tập dữ liệu kiểm tra. Cuối cùng, chúng ta chuẩn bị một tệp CSV để gửi, bao gồm các dự đoán doanh số cho các ngày cụ thể.

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, PolynomialFeatures
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV

In [2]:
# Load the datasets
train_df = pd.read_csv("E:/Jupyter NoteBook/XGBoost2/train.csv")
stores_df = pd.read_csv("E:/Jupyter NoteBook/XGBoost2/stores.csv")
oil_df = pd.read_csv("E:/Jupyter NoteBook/XGBoost2/oil.csv")
holidays_df = pd.read_csv("E:/Jupyter NoteBook/XGBoost2/holidays_events.csv")
transactions_df = pd.read_csv("E:/Jupyter NoteBook/XGBoost2/transactions.csv")

# Preprocessing


In [3]:
# Convert date columns to datetime objects
train_df['date'] = pd.to_datetime(train_df['date'])
oil_df['date'] = pd.to_datetime(oil_df['date'])
holidays_df['date'] = pd.to_datetime(holidays_df['date'])
transactions_df['date'] = pd.to_datetime(transactions_df['date'])

# Merge train data with store, oil, and transactions data
train_df = train_df.merge(stores_df, on='store_nbr', how='left')
train_df = train_df.merge(oil_df, on='date', how='left')
train_df = train_df.merge(transactions_df, on=['date', 'store_nbr'], how='left')

# Encode categorical features
le = LabelEncoder()
train_df['city'] = le.fit_transform(train_df['city'])
train_df['state'] = le.fit_transform(train_df['state'])
train_df['type'] = le.fit_transform(train_df['type'])

# Create holiday features
holidays_df['holiday'] = 1
train_df = train_df.merge(holidays_df[['date', 'holiday']], on='date', how='left')
train_df['holiday'].fillna(0, inplace=True)

# One-hot encode the 'family' column
ohe_family = OneHotEncoder(sparse=False, dtype=int)
family_encoded = ohe_family.fit_transform(train_df[['family']])
family_columns = ohe_family.get_feature_names_out(['family'])
family_df = pd.DataFrame(family_encoded, columns=family_columns, index=train_df.index)
train_df = pd.concat([train_df, family_df], axis=1)

# Drop the original 'family' column
train_df.drop('family', axis=1, inplace=True)

# Create lag features for oil prices
lags = [1, 7, 14]
for lag in lags:
    train_df[f'oil_lag_{lag}'] = train_df['dcoilwtico'].shift(lag)

# Create a feature for payday (every 15th and last day of the month)
train_df['payday'] = ((train_df['date'].dt.day == 15) | (train_df['date'].dt.is_month_end)).astype(int)

# Add date, month, year as features
train_df['day'] = train_df['date'].dt.day
train_df['month'] = train_df['date'].dt.month
train_df['year'] = train_df['date'].dt.year

# Add weekdays as one hot encoded features
train_df['weekday'] = train_df['date'].dt.day_name()
ohe = OneHotEncoder(sparse=False, dtype=int)
weekday_encoded = ohe.fit_transform(train_df[['weekday']])
weekday_columns = ohe.get_feature_names_out(['weekday'])
weekday_df = pd.DataFrame(weekday_encoded, columns=weekday_columns, index=train_df.index)
train_df = pd.concat([train_df, weekday_df], axis=1)

# Drop the original 'family' column
train_df.drop('weekday', axis=1, inplace=True)


# Drop rows with missing values
train_df.dropna(inplace=True)

# Add polynomial features
poly_features = ['day', 'month', 'year', 'oil_lag_1', 'oil_lag_7', 'oil_lag_14']
poly = PolynomialFeatures(degree=2, include_bias=False)
train_df_poly = poly.fit_transform(train_df[poly_features])


C:\Users\tk251\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\tk251\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [4]:
train_df.head()

,id,date,store_nbr,sales,onpromotion,city,state,type,cluster,dcoilwtico,...,day,month,year,weekday_Friday,weekday_Monday,weekday_Saturday,weekday_Sunday,weekday_Thursday,weekday_Tuesday,weekday_Wednesday
1796,1796,2013-01-02,1,3.0,0,18,12,3,13,93.14,...,2,1,2013,0,0,0,0,0,0,1
1797,1797,2013-01-02,1,0.0,0,18,12,3,13,93.14,...,2,1,2013,0,0,0,0,0,0,1
1798,1798,2013-01-02,1,0.0,0,18,12,3,13,93.14,...,2,1,2013,0,0,0,0,0,0,1
1799,1799,2013-01-02,1,0.0,0,18,12,3,13,93.14,...,2,1,2013,0,0,0,0,0,0,1
1800,1800,2013-01-02,1,0.0,0,18,12,3,13,93.14,...,2,1,2013,0,0,0,0,0,0,1


# Train Test Split

In [5]:
# Split the data into train and test sets
train_data, test_data = train_test_split(train_df, test_size=0.2, random_state=42, shuffle=True)

# Define the features and target variable
X_train = train_data.drop(['date', 'store_nbr', 'sales', 'dcoilwtico'], axis=1)
y_train = train_data['sales']
X_test = test_data.drop(['date', 'store_nbr', 'sales', 'dcoilwtico'], axis=1)
y_test = test_data['sales']

# Scale the features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
from sklearn.decomposition import PCA

# Apply PCA
pca = PCA(n_components=0.95)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# **XGBoost** & **ARIMA** Compared


In [8]:
from statsmodels.tsa.arima.model import ARIMA

# Define the XGBoost model
model_xg = XGBRegressor(
    n_estimators=100,
    max_depth=6,
    min_child_weight=1,
    learning_rate=0.1,
    subsample=0.8,
    colsample_bytree=0.8,
    tree_method='hist' 
)

# Define the ARIMA model
model_ar = ARIMA(y_train, order=(2, 0, 0))

# Fit the models
model_xg.fit(X_train, y_train)
model_ar = model_ar.fit()  

# Make predictions
predictions_xg = model_xg.predict(X_test)
predictions_ar = model_ar.predict(start=len(y_train), end=len(y_train) + len(y_test) - 1)  

# Calculate the mean squared error
mse_xg = mean_squared_error(y_test, predictions_xg)
mse_ar = mean_squared_error(y_test, predictions_ar)

C:\Users\tk251\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\tk251\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\tk251\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
C:\Users\tk251\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\tk251\AppData\Local

In [9]:
print(mse_xg)
print(mse_ar)

102475.99439151309
1011636.7909510806


# LSTM Results

LSTM is a type of Recurrent Neural Network (RNN) that is very effective in predicting time series data because it can remember patterns over the time period.

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Reshape input to be 3D [samples, timesteps, features]
X_train_lstm = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

In [11]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)


In [12]:
model_lstm = Sequential()
model_lstm.add(LSTM(50, activation='tanh', recurrent_activation='sigmoid', recurrent_dropout=0, unroll=False, use_bias=True, input_shape=(X_train_lstm.shape[1], X_train_lstm.shape[2])))
model_lstm.add(Dense(1))
model_lstm.compile(loss='mae', optimizer='adam')

history = model_lstm.fit(X_train_lstm, y_train, epochs=5, batch_size=72,
                         validation_data=(X_test_lstm, y_test), verbose=2, shuffle=False)

# make a prediction
predictions_lstm = model_lstm.predict(X_test_lstm)

# flatten predictions
predictions_lstm = predictions_lstm.flatten()

# calculate the mean squared error
mse_lstm = mean_squared_error(y_test, predictions_lstm)


Epoch 1/5
21443/21443 - 115s - loss: 275.7538 - val_loss: 236.5224 - 115s/epoch - 5ms/step
Epoch 2/5
21443/21443 - 109s - loss: 216.1333 - val_loss: 200.0111 - 109s/epoch - 5ms/step
Epoch 3/5
21443/21443 - 105s - loss: 188.1707 - val_loss: 178.2664 - 105s/epoch - 5ms/step
Epoch 4/5
21443/21443 - 110s - loss: 171.1525 - val_loss: 164.9102 - 110s/epoch - 5ms/step
Epoch 5/5
21443/21443 - 114s - loss: 160.1946 - val_loss: 155.6198 - 114s/epoch - 5ms/step
12062/12062 [==============================] - 37s 3ms/step


In [13]:
print(mse_lstm)

460182.6288739181


# Prophet results

Prophet is an open-source library developed by Facebook that is specifically designed for forecasting time series data. It's robust to missing data, shifts in the trend, and large outliers.



# Submission

In [14]:
scaler = MinMaxScaler()
scaler.fit(y_train.values.reshape(-1, 1))

MinMaxScaler()

In [15]:
# Choose the model with the lowest mean squared error
if min(mse_xg, mse_ar, mse_lstm) == mse_xg:
    model = model_xg
elif min(mse_xg, mse_ar, mse_lstm) == mse_ar:
    model = model_ar
else:
    model = model_lstm

# Make predictions using the chosen model
if model == model_lstm:
    X_test_lstm = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
    predictions = model.predict(X_test_lstm)
else:
    predictions = model.predict(X_test)

# Inverse transform the predictions
predictions = predictions.reshape(-1, 1)
predictions = scaler.inverse_transform(predictions)

# Create a submission DataFrame
submission = pd.DataFrame({"sales": predictions.flatten()})
# Assume that 'id' values are a range from 1 to the length of predictions
submission = pd.DataFrame({"id": range(1, len(predictions) + 1), "sales": predictions.flatten()})

# Save the submission DataFrame as a CSV file
submission.to_csv("submission.csv", index=False)


In [16]:
predictions

array([[ 2397228. ],
       [13189131. ],
       [-3814958.2],
       ...,
       [ 8697059. ],
       [19096656. ],
       [ -343371.1]], dtype=float32)

In [17]:
submission.head(-10)

,id,sales
0,1,2.397228e+06
1,2,1.318913e+07
2,3,-3.814958e+06
3,4,3.417156e+06
4,5,3.509830e+07
...,...,...
385949,385950,5.982828e+05
385950,385951,9.789154e+07
385951,385952,-2.461750e+06
385952,385953,8.590516e+07
